In [ ]:
%aiida

In [ ]:
from aiidalab_widgets_base import  StructureBrowserWidget
from apps.surfaces.widgets.viewer_details import ViewerDetails
from IPython.display import display, clear_output, HTML
import nglview
from ase import Atoms
from ase.data import covalent_radii, vdw_radii
from ase.neighborlist import NeighborList
import ase.neighborlist
import ase.build
from apps.surfaces.widgets import analyze_structure
import nglview
import numpy as np
from numpy.linalg import norm
import ipywidgets as ipw
style = {'description_width': '120px'}
style1 = {'description_width': '120px'}
style2 = {'description_width': '40px'}
layout = {'width': '20%'}
layout1 = {'width': '40%'}
layout2 = {'width': '15%'}


In [ ]:
def on_struct_select(c):
    global details, atoms,original_atoms,transformations
    if not struct_browser.results.value:
        return
    transformations=[]
    original_atoms=struct_browser.results.value.get_ase()
    atoms=original_atoms.copy()
    on_struct_change()
        
def on_struct_change():
    global details, atoms,original_atoms ,cov_radii,nl
    cov_radii = [covalent_radii[a.number] for a in atoms]
    nl = NeighborList(cov_radii, bothways = True, self_interaction = False)
    nl.update(atoms)
    details=analyze_structure.analyze(atoms)
    msg=details['summary']
    viewer_widget.setup(atoms,details=details)
    
    with mol_ids_info_out:
        clear_output()
        print(msg)


def str2vec(s):
    return np.array(list(map(float,s.split())))
    
def vec2str(v):
    return str(round(v[0],2))+ ' ' + str(round(v[1],2))+ ' ' +str(round(v[2],2))

def sel2com():
    global atoms
    selection=viewer_widget.selection
    if len(selection)==0:
        com=[0,0,0]
    else:
        com=atoms[selection].get_center_of_mass()
    return com

def axis_from_points():
    normal=str2vec(axis_p2.value)
    normal=normal - str2vec(axis_p1.value)
    norma=norm(normal)
    return normal/norma
        
        
def clear_selection(c):
    on_struct_change()
    
def def_selection(c):
    selection=viewer_widget.selection
    if len(selection)==0:
        selected.value=''
    else:
        selected.value=analyze_structure.mol_ids_range(viewer_widget.selection)     

def def_point(c):
    com=sel2com()
    point.value=vec2str(com)
    
def def_axis_p1(c):
    com=sel2com()
    axis_p1.value=vec2str(com) 
    
def def_axis_p2(c):
    global atoms
    selection=viewer_widget.selection
    if len(selection)==0:
        com=[0,0,1]
    else:
        com=atoms[selection].get_center_of_mass()
    axis_p2.value=vec2str(com)    
                
def string_range_to_list(a):
    singles=[int(s) -1 for s in a.split() if s.isdigit()]
    ranges = [r for r in a.split() if '..' in r]
    for r in ranges:
        t=r.split('..')
        to_add=[i-1 for i in range(int(t[0]),int(t[1])+1)]
        singles+=to_add
    return sorted(singles)

def translate_dr(c): 
    global transformations    

    rho=axis_from_points()*dr.value
    dxyz.value=vec2str(rho)
    transformations.append([selected.value,'translate',dxyz.value])
    modify_structure()

def translate_dxdydz(c):
    global transformations
    
    d=str2vec(dxyz.value)
    norma=norm(d)
    dr.value=round(norma,2)
    if norma < 0.0001:
        norma=1
        dr.value=norma
    normal=np.array(d)/norma
    axis_p1.value='0 0 0'
    axis_p2.value=vec2str(normal)
    transformations.append([selected.value,'translate',dxyz.value])
    modify_structure()
        
def rotate(c):
    global transformations
    normal=axis_from_points()
    transformations.append([selected.value,'rotate',phi.value,vec2str(normal),point.value])
    modify_structure()
    
def undo(c):
    global transformations
    
    if len(transformations)>0:
        transformations=transformations[0:-1]
    modify_structure()
    
def mod_element(c):
    global transformations
    
    transformations.append([selected.value,'modify',element.value])
    modify_structure()
    
def add(c):
    global transformations
    rho=axis_from_points()*dr.value  
    position=str2vec(point.value) + rho    
    transformations.append([selected.value,'add',add_list.value,position])
    modify_structure()
    
def remove(c):
    global transformations   
    transformations.append([selected.value,'remove'])
    modify_structure()
    selected.value = ""
    viewer_widget.reset_selection()
        
    
def modify_structure():
    global transformations,atoms,details,nl,original_atoms
    atoms=original_atoms.copy()
    #TRANSL [selected.value,'translate',dxyz.value]
    #ROT    [selected.value,'rotate',phi.value,vec2str(normal),point.value]
    #MOD    [selected.value,'modify',element.value]
    #ADD    [selected.value,'add',add_list.value,position]
    #REMOVE [selected.value,'remove']
    for op in transformations:
        #update neigh list after each operation        
        cov_radii = [covalent_radii[a.number] for a in atoms]
        nl = NeighborList(cov_radii, bothways = True, self_interaction = False)
        nl.update(atoms)
        
        selection=string_range_to_list(op[0])
        if op[1]=='rotate':
            c=atoms[selection].copy()
            vec=str2vec(op[3])
            center=str2vec(op[4])
            c.rotate(op[2], v=vec, center=center, rotate_cell=False)
            atoms.positions[selection]=c.positions
        elif op[1]=='translate':
            atoms.positions[selection]+=np.array(str2vec(op[2]))
            
        #####CHANGE ELEMENT    
        elif op[1]=='modify':
            for i in selection:
                atoms[i].symbol=op[2]
            
            
        #####REMOVE atoms
        elif op[1]=='remove':
            del[atoms[selection]]  
            
        #####ADD atoms   
        elif op[1]=='add':
            if op[2]=='H':
                if len(selection)==1:
                    idx=selection[0]
                    dCH=1.10 #1.1 for single H
                    vec = np.zeros(3)
                    indices, offsets = nl.get_neighbors(idx)
                    for i, offset in zip(indices, offsets):
                        if atoms[i].symbol=='C':
                            vec += -atoms[idx].position +(atoms.positions[i] + np.dot(offset, atoms.get_cell()))
                    vec = -vec/norm(vec)*dCH
                    vec += atoms[idx].position
                    vec1 = vec
                    first_h  = ase.Atom('H',vec1)
                    atoms.append(first_h)
            elif op[2]=='2H':
                if len(selection)==1:
                    idx=selection[0]
                    dCH=0.66 #1.1 for single H
                    vecz=np.array((0.00,0.00,0.88))
                    vec = np.zeros(3)
                    indices, offsets = nl.get_neighbors(idx)
                    for i, offset in zip(indices, offsets):
                        if atoms[i].symbol=='C':
                            vec += -atoms[idx].position +(atoms.positions[i] + np.dot(offset, atoms.get_cell()))
                    vec = -vec/norm(vec)*dCH
                    vec += atoms[idx].position
                    vec1 = vec + vecz
                    vec2 = vec - vecz
                    first_h  = ase.Atom('H',vec1)
                    second_h = ase.Atom('H',vec2)
                    atoms.append(first_h)
                    atoms.append(second_h)
            else:                
                atom_to_add=ase.Atom(op[2],op[3])
                atoms.append(atom_to_add)
    on_struct_change()
            
##STRUCTURE BROWSER
struct_browser = StructureBrowserWidget()
struct_browser.results.observe(on_struct_select, names='value')
        
###VIEWER
viewer_widget = ViewerDetails()

###TEXT description of structure
mol_ids_info_out = ipw.Output()    

###CONTROL BUTTONS
dxyz = ipw.Text(description='', value='0 0 0',style={'description_width': '70px'}, layout={'width': '14%'})


dr = ipw.BoundedFloatText(description='', value=1, min=0, 
                                        max=2, step=0.1,style=style2, layout=layout2)

phi = ipw.BoundedFloatText(description='', value=0, min=-90, 
                                        max=90, step=5,style=style2, layout=layout2)

axis_p1 = ipw.Text(description='axis P1', value='0 0 0',style={'description_width': '70px'}, layout={'width': '14%'})
axis_p2 = ipw.Text(description='axis P2', value='0 0 1',style={'description_width': '70px'}, layout={'width': '14%'})

point = ipw.Text(description='point', value='0 0 0',style={'description_width': '70px'}, layout={'width': '14%'})


selected = ipw.Text(description='atoms to modify', value='1..4',style=style1, layout=layout1)


btn_move_dx = ipw.Button(description='MOVE_XYZ:',
        layout={'width': '10%'})
btn_move_dx.on_click(translate_dxdydz)

btn_move_dr = ipw.Button(description='MOVE_DR:',
        layout={'width': '10%'})
btn_move_dr.on_click(translate_dr)

btn_rotate = ipw.Button(description='ROTATE PHI:',
        layout={'width': '10%'})
btn_rotate.on_click(rotate)

btn_undo = ipw.Button(description='UNDO',button_style='warning',
        layout={'width': '10%'})
btn_undo.on_click(undo)

btn_modify = ipw.Button(description='Modify in:',
        layout={'width': '12%'})
btn_modify.on_click(mod_element)

btn_add = ipw.Button(description='ADD:',
        style={'description_width': '120px'},layout={'width': '12%'})
btn_add.on_click(add)

btn_remove = ipw.Button(description='REMOVE ATOMS',button_style='danger',
        style={'description_width': '120px'},layout={'width': '18%'})
btn_remove.on_click(remove)

btn_def_p = ipw.Button(description='from sel.',button_style='info',
        layout={'width': '9%'})
btn_def_p.on_click(def_point)

btn_def_a1 = ipw.Button(description='from sel.',button_style='info',
        layout={'width': '9%'})
btn_def_a1.on_click(def_axis_p1)

btn_def_a2 = ipw.Button(description='from sel.',button_style='info',
        layout={'width': '9%'})
btn_def_a2.on_click(def_axis_p2)

btn_def_selection = ipw.Button(description='get IDs from highlighted',button_style='info',
        layout={'width': '18%'})
btn_def_selection.on_click(def_selection)

btn_clear_selection = ipw.Button(description='clear viewer',
        layout={'width': '12%'})
btn_clear_selection.on_click(clear_selection)

element=ipw.Dropdown(description="",
                            options=["H","C","N","O","Br","B"],
                            value="H",style={'description_width': '4px'}, layout={'width': '5%'}
                            )
add_list=ipw.Dropdown(description="",
                            options=["H","2H","3H","C"],
                            value="H",style={'description_width': '4px'}, layout={'width': '5%'}
                            )

#btn_spin_up = ipw.Button(description='spin U',
#        layout={'width': '8%'}
#btn_spin_down = ipw.Button(description='spin D',
#        layout={'width': '8%'}
#btn_set_spin_up.on_click(spinup)
#btn_set_spin_down.on_click(spindown)

In [ ]:
display(ipw.VBox([
    struct_browser,
    ipw.VBox([viewer_widget]),
    ipw.HBox([selected,btn_def_selection,btn_clear_selection,btn_undo,]),
    ipw.HBox([btn_rotate, phi, btn_move_dr, dr , btn_move_dx, dxyz]),
    ipw.HBox([axis_p1,btn_def_a1,axis_p2,btn_def_a2,point,btn_def_p]),
    ipw.HBox([btn_modify,element,btn_add,add_list,btn_remove]),
    mol_ids_info_out
    ]))